<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js113_SEIRinfectionModelDEQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [3]:
#@title js113_SEIRinfectionModelDEQ / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023 Mitsuru Ikeuchi
#  home page: http://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.14 created,  last updated on 2023.12.14
#


# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js113_SEIRinfectionModelDEQ</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js113_SEIRinfectionModelDEQ
//    Copyright(C) 2020-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2020.12.22 created, last updated on 2021.01.21
//    ver 0.0.1  2021.10.17 v1, last updated on 2021.10.17
//    ver 0.0.2  2021.11.08 v2, last updated on 2021.11.08
//    ver 0.0.3  2023.05.19 v3, last updated on 2023.09.13
//
// -------------------- SEIR infection model - Finite Difference Method
//
//    S-E-I-R model
//
//      [S] --- beta*I ---> [E] --- sigma ---> [I]--- gamma ---> [R]
//
//
//    S-E-I-R model -- differential equation
//      dS/dt = -beta*S*I
//      dE/dt = beta*S*I - sigma*E
//      dI/dt = sigma*E - gamma*I
//      dR/dt = gamma*I
//    where
//      S: susceptible
//      E: exposed
//      I: infectious
//      R: removed (recover + dead)
//      N = S + E + I + R = constant
//      beta: infection rate (1/day)
//      sigma: incidence rate (1/day) ~ 1/(mean incubation period)
//      gamma: recovery rate (1/day) ~ 1/(mean infected period)
//      mue: birth rate = death rate (1/day) ~ 1/(mean life period)
//
//    basic reproduction number R0
//      R0 = beta*S0/gamma
//     (R0 ~ 2~3 for influenza)  -- infection model
//
//
// --------------------
*/

const SEIRinfectionModel = (function(){ // ====================  SEIRinfectionModel Module  ====================

	const g_nMax = 360*3*20;			// array max
	const g_dt = 1.0/20.0;				// (day) time step g_dt

	let g_beta = 0.00025;				// (1/day) infection rate
	let g_sigma = 0.2;					// (1/day) incidence rate ~ 1/(mean incubation period) = 1/(5days)
	let g_gamma = 0.1;					// (1/day) recovery rate ~ 1/(mean infected period) = 1/(10days)
	let g_removedRatio = 0.0;			//

	let g_pN0 = 1000.0;					// (person) total = S+I+R at t=0
	let g_pE0 = 1.0;					// (person) exposed at t=0
	let g_pI0 = 0.0;					// (person) infectius at t=0
	let g_pS0 = g_pN0 - g_pI0;			// (person) susceptible at t=0
	let g_pR0 = 0.0;					// (person) removed at t=0

	const g_pS = dim1( g_nMax );		// (person) susceptible
	const g_pE = dim1( g_nMax );		// (person) exposed
	const g_pI = dim1( g_nMax );		// (person) infectius
	const g_pR = dim1( g_nMax );		// (person) removed
	const g_pN = dim1( g_nMax );		// (person) total = S+E+I+R

	function dim1( n ) {
		return new Float64Array( n );
	}


	// --------------------  set SEIR function  --------------------

	function setSEIRfunc( v_pN0, v_pI0, v_beta, v_sigma, v_gamma, v_removedRatio  ) {
		const nn=g_nMax, dt=g_dt;

		// set gloval
		g_beta = v_beta;
		g_sigma = v_sigma;
		g_gamma = v_gamma;

		g_pN0 = v_pN0;
		g_pE0 = 0.0;
		g_pI0 = v_pI0;
		g_pS0 = g_pN0 - g_pI0 - g_pN0*v_removedRatio;
		g_pR0 = g_pN0*v_removedRatio;

		// set initial value
		const beta = g_beta;
		const sigma = g_sigma;
		const gamma = g_gamma;
		let pS = g_pS0;
		let pE = g_pE0;
		let pI = g_pI0;
		let pR = g_pR0;
		let pN = pS + pE + pI + pR;

		// solve difference equation --> set S(t),E(t),I(t),R(t),N(t),  t=0...nMax
		for (let i=0; i<nn; i++) {
			g_pS[i] = pS;
			g_pE[i] = pE;
			g_pI[i] = pI;
			g_pR[i] = pR;
			g_pN[i] = pS + pE + pI + pR;

			// dS/dt = -beta*S*I
			const dSvdt = -beta*pS*pI;

			// dE/dt = beta*S*I - sigma*E
			const dEvdt = beta*pS*pI - sigma*pE;

			// dI/dt = sigma*E - gamma*I
			const dIvdt = sigma*pE - gamma*pI;

			// dR/dt = gamma*I
			const dRvdt = gamma*pI;

			pS = pS + dSvdt*dt;
			pE = pE + dEvdt*dt;
			pI = pI + dIvdt*dt;
			pR = pR + dRvdt*dt;
		}
	}


	// --------------------  utility  --------------------

	function maxInfectiusIndex() {
		const nn=g_nMax;

		let im = 0, m = g_pI[0];
		for(let i=0; i<nn; i++) {
			if (g_pI[i]>m) {
				m = g_pI[i];
				im = i;
			}
		}

		return im;
	}

	function  reproductionNumber() {
		return g_beta*g_pS[0]/g_gamma;
	}


	// --------------------  public  --------------------

	return {
		setSEIRfunc:	setSEIRfunc,	// setSEIRfunc( v_pN0, v_pI0, v_beta, v_sigma, v_gamma, v_removedRatio )

		getSysParam:	function() { return [ g_nMax, g_pN0, g_dt, reproductionNumber() ]; },
		getS:			function(i) { return g_pS[i]; },
		getE:			function(i) { return g_pE[i]; },
		getI:			function(i) { return g_pI[i]; },
		getR:			function(i) { return g_pR[i]; },
		getN:			function(i) { return g_pN[i]; },
		getSEIRN:		function(i) { return [ g_pS[i], g_pE[i], g_pI[i], g_pR[i], g_pN[i] ]; },
		getImax:		function() {  const iMax = maxInfectiusIndex(); return [ iMax, g_pI[iMax] ]; },
	};

})(); // ====================  SEIRinfectionModel end  ====================


const js113 = (function(){ // ====================  js Module  ====================

	const theModule = SEIRinfectionModel;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_pN0 = 1000.0;			// (person) nPop = susceptible + infectious + removed
	let v_pI0 = 1.0;			// (person) infectious = 1
	let v_beta = 0.00025;		// infection rate (1/day/infectious)
	let v_sigma = 0.2;			// incidence rate (1/day) ~ 1/(mean incubation period) = 1/(5days)
	let v_gamma = 0.1;			// recovery rate (1/day) ~ 1/(mean infected period)
	let v_removedRatio = 0.0;

	let p_nMax, p_pN0, p_dt, p_reproductionNumber; // <-- theModule.getSysParam()

  let ssList = [];
  let eeList = []
  let iiList = [];
  let rrList = [];
  let nnList = [];

	let dispMode = 0; 			// no use
	let timeRange = 360;		// (day)
	let resetFlag = true;
	let outputFlag = false;

  let breakFlag = false;
  let dataFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d', { willReadFrequently:true } );
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			outputFlag = false;
			theModule.setSEIRfunc( v_pN0, v_pI0, v_beta, v_sigma, v_gamma, v_removedRatio );
			[ p_nMax, p_pN0, p_dt, p_reproductionNumber ] = theModule.getSysParam();
      dataFlag = true;
		}

		draw( ctx, dispMode, timeRange, outputFlag );

    if ( dataFlag ) setPopData();

		requestAnimationFrame(animate);
	}

  function setPopData() {
    ssList = [];
    eeList = [];
    iiList = [];
    rrList = [];
    nnList = [];
    for (let i=0; i<timeRange; i++) {
      const j = i*20;
      let pS, pE, pI, pR, pN;
      [ pS, pE, pI, pR, pN ] = theModule.getSEIRN(j);
      ssList[i] = pS;
      eeList[i] = pE;
      iiList[i] = pI;
      rrList[i] = pR;
      nnList[i] = pN;
    }
	}


	//--------------------  draw  --------------------

	const gColor = { S:"#0088ff", Ex:"#ffff00", I:"#ff0000", R:"#00ff00", N:"#ff00ff", text:"#888888", grid:"#666666" };

	function draw( ctx, dispMode, timeRange, outputFlag ) {
		const xBoxSize = 400, yBoxSize = 400, xp=60, yp=60;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if (dispMode==0) {
			plotSEIR(ctx, xp,yp,timeRange);
		} else if (dispMode==1) {
			logPlotSEIR(ctx, xp,yp,timeRange);
		}


		const iMax = theModule.getImax()[0];
		let pS, pE, pI, pR, pN;
			[ pS, pE, pI, pR, pN ] = theModule.getSEIRN(iMax);
		ctx.fillStyle = "#888888";
		ctx.fillText(`infectius max = ${pI.toFixed(1)} at ${(iMax*p_dt).toFixed(2)} (day),`
						+ `  S+E+I+R = ${(pS+pE+pI+pR).toFixed(2)}`, 10, yCanvasSize-25);
		ctx.fillText(`beta = ${v_beta.toFixed(5)},  sigma = ${v_sigma.toFixed(2)},  gamma = ${v_gamma.toFixed(2)},`
						+ `  R0 = ${p_reproductionNumber.toFixed(2)}`, 10, yCanvasSize-5);

		if ( outputFlag ) outputTextArea(1,timeRange);
	}

	function plotSEIR(ctx, xp,yp,timeRange) {
		const mag = 300.0/p_pN0, m=Math.floor(timeRange/360/p_dt);

		//drawGrid(xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color)
		drawGrid(ctx, xp,yp,360,300,30,30,"0",timeRange+"(day)","0","1000", gColor.grid );

		drawText(ctx, "Susceptible", xp+20, yp-5, gColor.S );
		drawText(ctx, "Exposed", xp+70, yp-25, gColor.Ex );
		drawText(ctx, "Infectious", xp+120, yp-5, gColor.I );
		drawText(ctx, "Removed", xp+220, yp-5, gColor.R );
		drawText(ctx, "Population", xp+160, yp-25, gColor.N );
		drawText(ctx, "plot SEIR", xp+100, yp+360, gColor.text );

		plotFunc( ctx, xp, yp, m, theModule.getS, mag, gColor.S );
		plotFunc( ctx, xp, yp, m, theModule.getE, mag, gColor.Ex );
		plotFunc( ctx, xp, yp, m, theModule.getI, mag, gColor.I );
		plotFunc( ctx, xp, yp, m, theModule.getR, mag, gColor.R );
		plotFunc( ctx, xp, yp, m, theModule.getN, mag, gColor.N );
	}

	function plotFunc(ctx, xp,yp,m,pFunc,mag,colr) {
		ctx.strokeStyle = colr;
		ctx.beginPath();
		for (let i=0; i<360; i++) {
			ctx.lineTo(xp+i, yp+300-mag*pFunc(i*m));
		}
		ctx.stroke();
	}

	function logPlotSEIR(ctx, xp,yp,timeRange) {
		const m=Math.floor(timeRange/360/p_dt);

		//drawGrid(xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color)
		drawGrid(ctx, xp,yp,360,300,30,50,"0",timeRange+"(day)","10^0","10^3", gColor.grid );

		drawText(ctx, "Susceptible", xp+20, yp-5, gColor.S );
		drawText(ctx, "Exposed", xp+70, yp-25, gColor.Ex );
		drawText(ctx, "Infectious", xp+120, yp-5, gColor.I );
		drawText(ctx, "Removed", xp+220, yp-5, gColor.R );
		drawText(ctx, "Population", xp+160, yp-25, gColor.N );
		drawText(ctx, "log plot SEIR", xp+100, yp+360, gColor.text );

		plotLog10Func(ctx, xp,yp,m,theModule.getS, gColor.S );
		plotLog10Func(ctx, xp,yp,m,theModule.getE, gColor.Ex );
		plotLog10Func(ctx, xp,yp,m,theModule.getI, gColor.I );
		plotLog10Func(ctx, xp,yp,m,theModule.getR, gColor.R );
		plotLog10Func(ctx, xp,yp,m,theModule.getN, gColor.N );
	}

	function plotLog10Func(ctx, xp,yp,m,pFunc,colr) {
		ctx.strokeStyle = colr;
		ctx.beginPath();
		for (let i=0; i<360; i++) {
			let p = pFunc(i*m);
			if (p<=0.0) p = 0.1;
			const logp = Math.log10(p);
			if (logp>-0.2) ctx.lineTo(xp+i, yp+300-100.0*logp);
		}
		ctx.stroke();
	}

	function drawGrid(ctx, xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color) {
		ctx.strokeStyle = color;
		ctx.fillStyle = color;
		ctx.strokeRect(xp,yp,xLen,yLen);

		for (let i=0; i<xLen; i+=xDiv) {
			drawLine(ctx, xp+i,yp, xp+i, yp+yLen,color);
		}
		for (let j=0; j<yLen; j+=yDiv) {
			drawLine(ctx, xp,yp+yLen-j, xp+xLen, yp+yLen-j,color);
		}
		ctx.fillText(x0Str, xp-5, yp+yLen+20);
		ctx.fillText(xMaxStr, xp+xLen-xMaxStr.length*4, yp+yLen+20);
		ctx.fillText(y0Str, xp-y0Str.length*8-12, yp+yLen+5);
		ctx.fillText(yMaxStr, xp-yMaxStr.length*8-12, yp+5);
	}

	 function drawLine( ctx, x1, y1, x2, y2, color ) {
	 	ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}

	function drawText( ctx, txt, x, y, color ) {
		ctx.fillStyle = color;
		ctx.fillText(txt, x, y);
	}


	// --------------------  data output  --------------------

	function outputTextArea(outputSW,timeRange) {
		// outputSW:  0:textarea clear,  1:output text data
		outputFlag = false;

		let str;

		if (outputSW==0) {
			str = "";

		} else if (outputSW==1) {
			let pS, pE, pI, pR, pN;
			[ pS, pE, pI, pR, pN ] = theModule.getSEIRN(0);
			str = `beta = ${v_beta.toFixed(5)},  sigma = ${v_sigma.toFixed(2)},  gamma = ${v_gamma.toFixed(2)} \n`;
				+ `R0 = ${p_reproductionNumber.toFixed(2)}\n`;
				+ `S(0) = ${pS.toFixed(1)},  E(0) = ${pE.toFixed(1)},  I(0) = ${pI.toFixed(1)},`
				+ `  R(0) = ${pR.toFixed(1)}\n \n`;

			str += ` day,  S(t),  E(t),  I(t),  R(t) \n`;
			for (let i=0; i<timeRange; i++) {
				const j = i*20;
				[ pS, pE, pI, pR, pN ] = theModule.getSEIRN(j);
				str += ` ${i},  ${pS.toFixed(1)},  ${pE.toFixed(1)},  ${pI.toFixed(1)},  ${pR.toFixed(1)}\n`;
			}
		}
		document.getElementById("outputText").innerHTML = str;
	}


	// --------------------  dom control  --------------------

	function setBeta() {
		const b = 0 + document.getElementById("range_beta").value;
		v_beta = 0.00001*b;
		document.getElementById("text_beta").innerHTML = " " + v_beta.toFixed(5);
		resetFlag = true;
	}

	function setSigma() {
		const s = 0 + document.getElementById("range_sigma").value;
		v_sigma = 0.005*s;
		document.getElementById("text_sigma").innerHTML =
			" " + v_sigma.toFixed(2) + " (1/ " + (1.0/v_sigma).toFixed(1) + " day)";
		resetFlag = true;
	}

	function setGamma() {
		const g = 0 + document.getElementById("range_gamma").value;
		v_gamma = 0.005*g;
		document.getElementById("text_gamma").innerHTML =
			" " + v_gamma.toFixed(2) + " (1/ " + (1.0/v_gamma).toFixed(1) + " day)";
		resetFlag = true;
	}

	function setRemovedRatio() {
		const r = 0 + document.getElementById("range_removedRatio").value;
		v_removedRatio = 0.01*r;
		document.getElementById("text_removedRatio").innerHTML = " " + v_removedRatio.toFixed(2);
		resetFlag = true;
	}

	function setDispMode() {
		dispMode = document.getElementById("slct_dispMode").selectedIndex;
	}

	function setTimeRange() {
		timeRange = setTimeRange.range[ 0 + document.getElementById("slct_timeRange").selectedIndex ];
	}
	setTimeRange.range = [180,360,720,1080];

	function getData() {
		outputFlag = true;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetBeta( beta ) {
		v_beta = beta;
		document.getElementById("text_beta").innerHTML = " " + v_beta.toFixed(5);
		resetFlag = true;
  }

  function pysetSigma( sigma ) {
		v_sigma = sigma;
		document.getElementById("text_sigma").innerHTML =
			" " + v_sigma.toFixed(2) + " (1/ " + (1.0/v_sigma).toFixed(1) + " day)";
		resetFlag = true;
  }

  function pysetGamma( gamma ) {
		v_gamma = gamma;
		document.getElementById("text_gamma").innerHTML =
			" " + v_gamma.toFixed(2) + " (1/ " + (1.0/v_gamma).toFixed(1) + " day)";
		resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ v_pN0, v_pI0, v_beta, v_sigma, v_gamma, v_removedRatio, p_reproductionNumber ];
  }

  function pygetPopData() {
    dataFlag = false;
    return [ ssList, eeList, iiList, rrList, nnList ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		// dom control
		setBeta:			setBeta,			// setBeta()
		setSigma:			setSigma,			// setSigma()
		setGamma:			setGamma,			// setGamma()
		setRemovedRatio:	setRemovedRatio,	// setRemovedRatio()

		setDispMode:		setDispMode,		// setDispMode()
		setTimeRange:		setTimeRange,		// setTimeRange()
		getData:			getData,			// getData()

    breakLoop: breakLoop, // breakLoop();
    pysetBeta: pysetBeta, // pysetBeta( beta )
    pysetSigma: pysetSigma, // pysetSigma( sigma )
    pysetGamma: pysetGamma, // pysetGamma( gamma )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ pN0, pI0, beta, sigma, gamma, removedRatio, R0 ]
    pygetPopData: pygetPopData, //() :return [ ssList, iiList, rrList, nnList ]
	};

})(); // ====================  js113 module end  ====================


const js = js113;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js113] SEIR infection model - solve finite difference equations</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<span>theme: S-E-I-R infection model</span>
<br>

<label>beta (infection rate ) =</label>
<input type="range" id="range_beta" min="1" max="100" value="25" step="1"
style="width:200px" oninput="js.setBeta()">
<label id="text_beta"> 0.00025</label> (1/day/I)
<br>

<label>sigma (incidence rate)=</label>
<input type="range" id="range_sigma" min="1" max="100" value="40" step="1"
style="width:200px" oninput="js.setSigma()">
<label id="text_sigma"> 0.20 (1/day)</label>
<br>

<label>gamma (recovery rate)=</label>
<input type="range" id="range_gamma" min="1" max="100" value="20" step="1"
style="width:200px" oninput="js.setGamma()">
<label id="text_gamma"> 0.10 (1/day)</label>
<br>

<label>vaccinated ratio =</label>
<input type="range" id="range_removedRatio" min="0" max="100" value="0" step="1"
style="width:200px" oninput="js.setRemovedRatio()">
<label id="text_removedRatio"> 0.00</label>
<br>

<label>dispMode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>plot SEIR</option><option >log plot SEIR</option></select>
    <span style="margin-right: 50px;"></span>
<label>time range:</label>
<select id="slct_timeRange" onChange="js.setTimeRange()">
<option >0 - 180</option><option selected>0 - 360</option><option >0 - 720</option>
<option >0 - 1080</option></select>
<br>

<button id="getData_button" onClick="js.getData()">output text data</button>
<br>

<textarea id="outputText" rows="6" cols="60" disabled></textarea>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get S-I-R population

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

print("-- get system parameter --")
[ pN0, pI0, beta, sigma, gamma, removedRatio, R0 ] = eval_js( 'js.pygetData({})'.format(0) )
print(f' N = {pN0}, I0 = {pI0}, beta = {beta:>10.6f}, sigma = {sigma:>10.6f}, gamma = {gamma:>10.6f}, reproduction number R0 = {R0:6.2f} ')

# get SIR data
print("-- get SIR data --")
[ ssList, eeList, iiList, rrList, nnList ] = eval_js('js.pygetPopData()')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

print(f'len(ssList) = {len(ssList)}, len(eeList) = {len(eeList)}, len(iiList) = {len(iiList)}, len(rrList) = {len(rrList)}, len(nnList) = {len(nnList)}, ')
print("-- end --")

In [ ]:
# save data

import numpy as np

print("-- save data : np.array([ spinorRe, spinorIm ]) --")
np_data = np.array([ ssList, eeList, iiList, rrList, nnList ])

np.save('js113_data.npy', np_data)

print("-- saved data shape :", np_data.shape, " --")

# The saved file can be viewed by clicking on the folder icon on the left edge of the notebook.
# You must download this file to save it permanently.
# Select Download from the '...'.T menu of the file to download it.

In [ ]:
# plot S-E-I-R

import matplotlib.pyplot as plt

plt.plot(ssList, 'b-')
plt.plot(eeList, 'm-')
plt.plot(iiList, 'r-')
plt.plot(rrList, 'g-')
plt.title('S-E-I-R Model')
plt.xlabel('time( day )')
plt.ylabel('population')
plt.grid()
plt.show()

In [ ]:
# Colab AI wrote:
# prompt: translate matplotlib in 'plot S-E-I-R' cell to plotly with legend

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ssList)), y=ssList, name='Susceptible'))
fig.add_trace(go.Scatter(x=np.arange(len(eeList)), y=eeList, name='Exposed'))
fig.add_trace(go.Scatter(x=np.arange(len(iiList)), y=iiList, name='Infected'))
fig.add_trace(go.Scatter(x=np.arange(len(rrList)), y=rrList, name='Removed'))
fig.update_layout(title='S-E-I-R Model', xaxis_title='time( day )', yaxis_title='population', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()


In [15]:
# Colab AI wrote:
# prompt: as '# save data'-cell, save np_data.T as  csv file

import numpy as np

np.savetxt('js113_data.csv', np_data.T, delimiter=',')
